In [6]:
import websocket
import json
import duckdb
from datetime import datetime

# Production WebSocket URL
WEBSOCKET_URL = "wss://socket.india.delta.exchange"

# DuckDB setup
conn = duckdb.connect("btc_futures_candles.duckdb")
conn.execute("""
CREATE TABLE IF NOT EXISTS btc_futures_candles (
    candle_start_time TIMESTAMP,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    last_updated TIMESTAMP,
    symbol VARCHAR
)
""")

def insert_candle(message_json):
    candle_time = datetime.fromtimestamp(message_json["candle_start_time"] / 1e6)
    last_updated = datetime.fromtimestamp(message_json["last_updated"] / 1e6)
    
    conn.execute("""
        INSERT INTO btc_futures_candles VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        candle_time,
        message_json["open"],
        message_json["high"],
        message_json["low"],
        message_json["close"],
        last_updated,
        message_json["symbol"]
    ))
    print(f"Inserted candle for {message_json['symbol']} at {candle_time}")

def on_error(ws, error):
    print(f"Socket Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print(f"Socket closed with status: {close_status_code} and message: {close_msg}")

def on_open(ws):
    print("Socket opened")
    subscribe(ws, "candlestick_1m", ["MARK:BTCUSD"])

def subscribe(ws, channel, symbols):
    payload = {
        "type": "subscribe",
        "payload": {
            "channels": [
                {
                    "name": channel,
                    "symbols": symbols
                }
            ]
        }
    }
    ws.send(json.dumps(payload))

last_candle_time = None

def on_message(ws, message):
    global last_candle_time
    message_json = json.loads(message)

    if message_json.get("type") == "candlestick_1m" and message_json.get("symbol") == "MARK:BTCUSD":
        candle_time = datetime.fromtimestamp(message_json["candle_start_time"] / 1e6)

        # Insert only if it's a new candle
        if last_candle_time != candle_time:
            insert_candle(message_json)
            last_candle_time = candle_time


if __name__ == "__main__":
    ws = websocket.WebSocketApp(
        WEBSOCKET_URL,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    ws.on_open = on_open
    ws.run_forever()


ModuleNotFoundError: No module named 'websocket'